In [1]:
!pip install -q accelerate
!pip install -q peft
!pip install -q bitsandbytes
!pip install -q trl

!pip install -q pandas
!pip install -q transformers
!pip install -q torch
!pip install -q datasets
!pip install -q flash_attn
#!pip -q install scikit-learn


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip i

In [2]:
!pip install -q sentence-transformers
!pip install -q faiss-gpu


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
import pandas as pd
import numpy as np
import os

import transformers

import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
    set_seed
)

from transformers import logging

from peft import LoraConfig, PeftModel
from trl import SFTTrainer

from tqdm import tqdm

# Don't Show Warning Messages
import warnings
warnings.filterwarnings('ignore')

print(f"CUDA Version {torch.version.cuda}")
print(f"Pytorch {torch.__version__}")
print(f"Transformers {transformers.__version__}")

CUDA Version 12.1
Pytorch 2.1.1+cu121
Transformers 4.45.2


In [16]:
#from kaggle_secrets import UserSecretsClient

#user_secrets = UserSecretsClient()
#HF_ACCESS_TOKEN = user_secrets.get_secret("HF_ACCESS_TOKEN")

HF_ACCESS_TOKEN = "YOUR_HUGGINGFACE_TOKEN"

In [17]:
# Check the type and quantity of GPUs
# Note: Kaggle GPUs don't support flash attention

if torch.cuda.is_available():
    print('Num CPUs:', os.cpu_count())
    print('Num GPUs:', torch.cuda.device_count())
    print('GPU Type:', torch.cuda.get_device_name(0))

    # Check if bf16 is suported
    print("bf16 supported:",torch.cuda.is_bf16_supported())

Num CPUs: 96
Num GPUs: 1
GPU Type: NVIDIA A40
bf16 supported: True


## Settings

In [18]:
# Paths to the two csv files containing the synthetic data
DATA_PATH_1 = 'exp18-cleaned_afrikaans_childrens_stories.csv'
DATA_PATH_2 = 'exp37-cleaned_afrikaans_childrens_stories.csv'

DEVICE = "cuda"

In [19]:
# Set logging level to suppress unnecessary output

from transformers import logging

# Suppress huggingface info messages and only show errors
logging.set_verbosity_error()

## Helper functions

In [20]:
def run_inference(moral_value):

    system_message = "You are an expert at writing South African children's stories that teach children moral values and life lessons. You will be given a moral value. Write a captivating children's story in South African Arikaans that includes the moral value."
    prompt =  f"Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{system_message}\n\n### Input:\n{moral_value}\n\n### Response:\n"

    # The tkenizer automatically adds a <bos> token
    inputs = tokenizer(prompt, return_tensors="pt")

    # Send the inputs to the device
    inputs = inputs.to(DEVICE)

    """
    # This will create deterministic outputs

    # Generate the outputs from prompt
    generate_ids = model.generate(**inputs,
                                  max_new_tokens=1024,
                                  do_sample=False,
                                  temperature=0.5,
                                  )
    """

    # Generate the outputs from prompt with sampling enabled
    generate_ids = model.generate(**inputs,
                                  max_new_tokens=1024,
                                  do_sample=True,   # Enable sampling for variability
                                  temperature=0.8,  # Adjust for more randomness
                                  top_k=50,         # Limit to top-k tokens
                                  top_p=0.95,       # Or use nucleus sampling (top-p)
                                  )

    # Decode the generated output
    generated_text = tokenizer.batch_decode(generate_ids,
                                        skip_special_tokens=False,
                                        clean_up_tokenization_spaces=False)[0]

    #print(generated_text)

    # Extract the response
    response = generated_text.split('### Response:')[1]
    # Remove the <eos> token
    response = response.replace('<eos>', "")
    # Remove leading and trailing spaces
    response = response.strip()


    return response, generated_text


## Load the data

In [21]:
# Load the first csv file
path = DATA_PATH_1
df1 = pd.read_csv(path)

# Load the second csv file
path = DATA_PATH_2
df2 = pd.read_csv(path)

# Concat the two dataframes
df_data = pd.concat([df1, df2], axis=0)

# Reset the index
df_data = df_data.reset_index(drop=True)

print(df_data.shape)

df_data.head(3)

(564, 8)


,input_for_prompt_gen,prompt,prompt_language,moral_value,first_draft,final_draft,changes_made,translation
0,\n You are an expert at creating prompts th...,Write a South African Afrikaans children's sto...,British English,Kindness – Showing care and consideration for ...,## Klein Karoo se Groot Hart\n\nKlein Karoo wa...,Klein Karoo se Groot Hart\n\nKlein Karoo was '...,"The original story was already quite good, but...",## Little Karoo's Big Heart\n\nLittle Karoo wa...
1,\n You are an expert at creating prompts th...,Skep 'n Suid-Afrikaanse kinderverhaal in natuu...,Afrikaans,Honesty – Valuing truth and integrity in words...,"## Klein Karoo se Groot Geheim\n\nKlein Karoo,...",Klein Karoo se Groot Bessie-Geheim\n\nKlein Ka...,The following changes were made to improve the...,## Little Karoo's Big Berry Secret\n\nLittle K...
2,\n You are an expert at creating prompts th...,Write a South African children's story in Afri...,British English,Courage – Encouraging bravery to face fears an...,## Klein Klein Leeutjie se Groot Moed\n\nKlein...,"Leo se Groot Moed\n\nKlein Leo, of Leon soos s...",The following changes were made to improve the...,"## Leo's Great Courage\n\nLittle Leo, or Leon ..."


## Format the text input

In [22]:
def create_text_col(row):

    # Note: The tokenizer automatically adds a <bos> token so
    # we don't need to add it here. We only add the <eos> token.

    # Get the story and the moral value
    response = row['final_draft'].rstrip()
    moral_value = row['moral_value'].rstrip()

    # Define the system message and the user input
    system_message = "You are an expert at writing South African children's stories that teach children moral values and life lessons. You will be given a moral value. Write a captivating children's story in South African Arikaans that includes the moral value."

    text =  f"Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{system_message}\n\n### Input:\n{moral_value}\n\n### Response:\n{response}<eos>"

    return text

df_data['text'] = df_data.apply(create_text_col, axis=1)

print(df_data.shape)

#df_data.head(3)

(564, 9)


In [23]:
# Print one formatted prompt

print(df_data.loc[1, 'text'])

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are an expert at writing South African children's stories that teach children moral values and life lessons. You will be given a moral value. Write a captivating children's story in South African Arikaans that includes the moral value.

### Input:
Honesty – Valuing truth and integrity in words and actions.

### Response:
Klein Karoo se Groot Bessie-Geheim

Klein Karoo, 'n klein bobbejaan met 'n piepklein stertjie, het 'n blinkrooi bessie gevind. Die bessie was so groot soos sy duim en so blink soos 'n robbe se oog. Hy het dit agter 'n groot klip weggesteek, net vir homself.

Sy ma, Ouma Karoo, het hom gevra: "Karoo, my klein mannetjie, het jy dalk 'n lekker bessie gevind vandag?"

Karoo het sy lippe styfgebyt. Hy wou die bessie vir homself hou. "Nee, Ouma," het hy gefluister, "ek het niks gevind nie."

Ouma K

## Get the num tokens

In [24]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-9b",
                                          trust_remote_code=True,
                                          use_auth_token=HF_ACCESS_TOKEN)


In [25]:
# Get the num tokens for each sample

def get_num_tokens(x):

    inputs = tokenizer(x, return_tensors="pt",
                       truncation=False).to('cpu')

    token_list = inputs['input_ids'][0]

    num_tokens = len(token_list)

    return num_tokens

# Get a count of input tokens
df_data['num_tokens'] = df_data['text'].apply(get_num_tokens)

print('Total:', df_data['num_tokens'].sum())
print('Max:', df_data['num_tokens'].max())
print('Min:', df_data['num_tokens'].min())

Total: 412599
Max: 936
Min: 543


## Convert to Huggingface dataset

In [26]:
from datasets import Dataset

# Select the columns we want to keep
cols = ['text']

# Convert the pandas DataFrame to a Hugging Face Dataset
hf_dataset_train = Dataset.from_pandas(df_data[cols])


## Initialize the quantized model and tokenizer

In [27]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from trl import SFTTrainer


MODEL_NAME = "google/gemma-2-9b"
ADAPTER_SAVE_PATH = f"gemma-2-9b-lora-adapter"
MERGED_MODEL_SAVE_PATH = f"gemma-2-9b-4bit-afrik-stories"
MAX_LEN = 1024

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    use_auth_token=HF_ACCESS_TOKEN,
    use_cache=False,
    trust_remote_code=True,
    quantization_config=bnb_config,
)


tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME,
                                         use_auth_token=HF_ACCESS_TOKEN)


model-00007-of-00008.safetensors:  20%|##        | 1.01G/4.96G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/2.38G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

### Fine tune the 4-bit quantized model

In [28]:

peft_config = {
    "r": 16,
    "lora_alpha": 32,
    "lora_dropout": 0.01,
    "bias": "none",
    "task_type": "CAUSAL_LM",
    "target_modules":["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
}

peft_conf = LoraConfig(**peft_config)


training_config = {
    "num_train_epochs": 2,
    "learning_rate": 2e-4,
    "lr_scheduler_type": "cosine",
    "warmup_ratio": 0.2,
    "optim": "paged_adamw_8bit",

    #"per_device_eval_batch_size": 1, # Note: Evaluation is done using only one GPU
    "per_device_train_batch_size": 2,
    "gradient_accumulation_steps": 2,
    "max_steps": -1,

    "do_eval": False,
    "evaluation_strategy": "no", #epoch #steps #no

    "logging_strategy": "steps",
    "logging_steps": 20,
    "log_level": "info",

    "save_strategy": "steps", #epoch #steps
    "save_steps": 100,
    "save_total_limit": 1,
    "overwrite_output_dir": True,
    "output_dir": "./checkpoint_dir",

    "gradient_checkpointing": True,
    "bf16": True,
    "seed": 1024,
    "report_to": "none",  # Disable logging to external systems like WandB
    "disable_tqdm": False  # Enable the progress bar
    }

train_conf = TrainingArguments(**training_config)


trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    args=train_conf,
    peft_config=peft_conf,
    train_dataset=hf_dataset_train,
    #eval_dataset=hf_dataset_val,
    max_seq_length=MAX_LEN,
    dataset_text_field="text",
    packing=False,
)


Map:   0%|          | 0/564 [00:00<?, ? examples/s]

Using auto half precision backend


In [29]:
# Run training
trainer.train()

# Save the adapter
trainer.save_model(ADAPTER_SAVE_PATH)

***** Running training *****
  Num examples = 564
  Num Epochs = 2
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 2
  Total optimization steps = 282
  Number of trainable parameters = 54,018,048


Step,Training Loss
20,1.465200
40,1.067600
60,1.006800
80,0.949200
100,0.928800
120,0.901000
140,0.893000
160,0.668500
180,0.642000
200,0.620700


Saving model checkpoint to ./checkpoint_dir/checkpoint-100
tokenizer config file saved in ./checkpoint_dir/checkpoint-100/tokenizer_config.json
Special tokens file saved in ./checkpoint_dir/checkpoint-100/special_tokens_map.json
Saving model checkpoint to ./checkpoint_dir/checkpoint-200
tokenizer config file saved in ./checkpoint_dir/checkpoint-200/tokenizer_config.json
Special tokens file saved in ./checkpoint_dir/checkpoint-200/special_tokens_map.json
Deleting older checkpoint [checkpoint_dir/checkpoint-100] due to args.save_total_limit
Saving model checkpoint to ./checkpoint_dir/checkpoint-282
tokenizer config file saved in ./checkpoint_dir/checkpoint-282/tokenizer_config.json
Special tokens file saved in ./checkpoint_dir/checkpoint-282/special_tokens_map.json
Deleting older checkpoint [checkpoint_dir/checkpoint-200] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to gemma-2-9b-lora-adapter
to

In [30]:
import gc

del model
del tokenizer

torch.cuda.empty_cache()
gc.collect()
gc.collect()

0

## Merge the full precision model and the fine tuned adapter

In [31]:
# Set logging level to suppress unnecessary Huggingface output

from transformers import logging

# This gets changed during fine tuning so
# we need to set it again.
logging.set_verbosity_error()

In [32]:
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    use_auth_token=HF_ACCESS_TOKEN,
    use_cache=False,
    trust_remote_code=True,
)


tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME,
                                         use_auth_token=HF_ACCESS_TOKEN)

# Merge the model and adapter
model = PeftModel.from_pretrained(base_model, ADAPTER_SAVE_PATH)
model = model.merge_and_unload()

# Load the tokenizer so it can be saved
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME,
                                         use_auth_token=HF_ACCESS_TOKEN
                                         )

# Save the model
model.save_pretrained(MERGED_MODEL_SAVE_PATH)

# Save the tokenizer
tokenizer.save_pretrained(MERGED_MODEL_SAVE_PATH)


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

('gemma-2-9b-4bit-afrik-stories/tokenizer_config.json',
 'gemma-2-9b-4bit-afrik-stories/special_tokens_map.json',
 'gemma-2-9b-4bit-afrik-stories/tokenizer.json')

In [33]:
import gc

del model
del tokenizer

torch.cuda.empty_cache()
gc.collect()
gc.collect()

0

## Run evaluation

In [34]:
# Load the fine tuned model
model = AutoModelForCausalLM.from_pretrained(
    MERGED_MODEL_SAVE_PATH,
    device_map='auto',
    torch_dtype=torch.bfloat16,
    local_files_only=True,
)


# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(MERGED_MODEL_SAVE_PATH)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

## Generate on story for each moral value

Generate 28 stories that we can manually review.

In [35]:
moral_value_list = [
"Kindness – Showing care and consideration for others, especially those in need.",
"Honesty – Valuing truth and integrity in words and actions.",
"Courage – Encouraging bravery to face fears and challenges.",

"Responsibility – Teaching accountability and the importance of fulfilling commitments.",
"Respect – Valuing other people’s rights, opinions, and differences.",
"Perseverance – Promoting resilience and the determination to keep going despite challenges.",

"Forgiveness – Encouraging letting go of grudges and offering second chances.",
"Gratitude – Emphasizing thankfulness for what one has and the kindness of others.",
"Teamwork – Highlighting the importance of working together and supporting one another.",

"Fairness – Teaching equity and justice, where everyone is treated equally.",
"Generosity – Encouraging giving to others, particularly those less fortunate.",
"Patience – Fostering calmness and the ability to wait without frustration.",

"Empathy – Helping children understand and care about the feelings of others.",
"Humility – Promoting modesty and recognizing the strengths and contributions of others.",
"Self-discipline – Teaching control over impulses and making responsible choices.",

"Gender Equality – Promoting equal rights and opportunities for all genders.",
"Religious Tolerance – Encouraging understanding and acceptance of different faiths and beliefs.",
"Strength in Diversity – Celebrating the value of different cultures, backgrounds, and perspectives.",

"Racial Equality – Fostering respect and equality among all races, promoting fairness and inclusion.",
"Environmental Stewardship – Encouraging responsibility for protecting the planet and its resources.",
"Compassion for Animals – Promoting kindness and care for all living creatures.",

"Peacefulness – Teaching peaceful conflict resolution and the importance of non-violence.",
"Gratitude for Simple Things – Encouraging appreciation of everyday moments and small acts of kindness.",
"Sharing – Teaching the value of sharing what we have with others.",

"Independence and Self-reliance – Encouraging confidence in one’s abilities and the importance of taking initiative.",
"Speaking Out Against Injustice – Instilling the courage to stand up against unfairness, discrimination, or wrongdoing. This value teaches children that they have the power and responsibility to challenge injustice, whether it is directed at them or others, and to advocate for fairness and equality in their communities.",
"Being On Time – Teaching the value of punctuality and respect for others’ time, reinforcing discipline and reliability.",
"Striving for Excellence – Instilling a mindset of doing one’s best and aiming for high standards in all pursuits, whether in school, hobbies, or personal goals.",
]

len(moral_value_list)

28

In [36]:
# Generate a story for each moral value

response_list = []

print_output = False

for i in tqdm(range(0, len(moral_value_list))):

    # Get the moral value
    moral_value = moral_value_list[i]

    # Run inference
    response, _ = run_inference(moral_value)

    response_list.append(response)

    if print_output == True:

        print(f"--{i}--")
        print(f"Moral_value: {moral_value}")
        print(f"Generated story: {response}")
        print()


100%|██████████| 28/28 [17:06<00:00, 36.65s/it]


In [37]:
# Put the elvaluation output into a dataframe

data = {
    'moral_value': moral_value_list,
    'generated_story': response_list,
}

# Create the dataframe
df_eval = pd.DataFrame(data)

# Add an id column
df_eval['id'] = df_eval.index

# Save the dataframe
path = "eval_results.csv"
df_eval.to_csv(path, index=False)

print(df_eval.shape)

df_eval.head(3)

(28, 3)


,moral_value,generated_story,id
0,Kindness – Showing care and consideration for ...,"Klein Karoo se Groot Hart\n\nKlein Karoo, 'n k...",0
1,Honesty – Valuing truth and integrity in words...,"Klein Kotie en die Rooi Appel\n\nKlein Kotie, ...",1
2,Courage – Encouraging bravery to face fears an...,Klein Leeutjie se Groot Moed\n\nKlein Leeutjie...,2


## Check spelling, grammar and fluency

Manually review each generated story to ensure that the Afrikaans spelling and grammar are correct.
Also check that the output is fluent.

In [38]:
# Print the generated stories

for i in range(0, len(df_eval)):

    moral_value = df_eval.loc[i, 'moral_value']
    generated_story = df_eval.loc[i, 'generated_story']


    print(f"--{i}--")
    print(f"Moral_value: {moral_value}")
    print(f"Generated story: {generated_story}")
    print()

--0--
Moral_value: Kindness – Showing care and consideration for others, especially those in need.
Generated story: Klein Karoo se Groot Hart

Klein Karoo, 'n klein bobbejaan met 'n stert so lank soos 'n reënboog, was 'n bietjie van 'n slegte seuntjie. Hy het graag gespot en gejaag, en ander bobbejane se kos gevat. Hy het glad nie gedink aan ander se gevoelens nie. Sy ma, 'n wyse ou bobbejaan met sagte oë, het hom al baie keer vermaan: "Karoo, my seun, 'n goeie hart is belangriker as 'n sterk arm." Maar Karoo het net sy skouertjies opgetrek en verder met sy speletjies voortgegaan.

Een warm middag, terwyl Karoo besig was om 'n ander bobbejaan se lekkerste mango te steel, het hy 'n ou, siek bobbejaantjie onder 'n doringboom gesien. Die bobbejaantjie was dun en swak, en sy oë was vol trane. Hy het na 'n stukkie broodjie gewag, maar sy klouetjies was te swak om dit op te tel.

Karoo het vir 'n oomblik stilgestaan. Hy het geweet hy moes die broodjie steel om dit vir homself te eet. Maar to

## Check for randomness

Check that, given the same input, the model always generates a different story.

In [39]:
# Print the generated stories

j = 7

for i in range(0, 3):

    moral_value = df_eval.loc[j, 'moral_value']

   # Run inference
    generated_story, _ = run_inference(moral_value)


    print(f"--{i}--")
    print('---Moral value---')
    print(moral_value)
    print()
    print('---Generated story---')
    print(generated_story)
    print()

--0--
---Moral value---
Gratitude – Emphasizing thankfulness for what one has and the kindness of others.

---Generated story---
Klein Mielie se Dankie-Boompies

Klein Mielie, ’n klein, bruin meisie met krulhare soos springmielies, het baie gehad. Sy het ’n warm huis met ’n ouma wat heerlike koekies gebak het. Sy het ’n ou, maar lieflike kat, Smous, en ’n tuin vol lekkers: ryp vye, soet perskes en sappige appels. Maar Klein Mielie was altyd ontevrede.  "Ek wil ’n nuwe fiets hê," het sy gekla. "En ’n pienk pop! En sjokolade vir ontbyt!"

Een dag het Ouma Mielie se gunstelingkoekies gebak, heerlike, warm kaneelkoekies. Mielie het dit geproe, maar sonder ’n woord gevra vir sjokolade. Toe sy die koekies wegwerp, het Ouma saggies gesê: "Mielie, kind, kyk mooi rond. Kyk na jou huis, na Smous, na die vrugte in die tuin. Is daar dalk iets waarvoor jy dankbaar kan wees?"

Mielie het stil geword en om haar gekyk. Sy het die sagte kombers op haar bed, die son wat deur die venster skyn en die sagt

## Check for originality

Check that the model is not outputing stories that it has memorized from the training data.


In [40]:
from sentence_transformers import SentenceTransformer

# Instantiate the model
embed_model = SentenceTransformer("all-MiniLM-L6-v2")

# Create a list of stories that will be vectorized
chunk_list = list(df_data['final_draft'])

# Convert the train set stories into embeddings
embeddings = embed_model.encode(chunk_list)

print(embeddings.shape)
print('Embedding length', embeddings.shape[1])


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(564, 384)
Embedding length 384


In [41]:
import faiss

embed_length = embeddings.shape[1]

index = faiss.IndexFlatL2(embed_length)

# Add the embeddings to the index
index.add(embeddings)

# Check the total number of embeddings in the index
index.ntotal

564

In [42]:
print(df_eval.loc[2, 'generated_story'])

Klein Leeutjie se Groot Moed

Klein Leeutjie, met sy sagte, goue mannetjie, was bang. Bang vir die donker, bang vir die wind wat deur die gras fluister, en bo alles, bang vir die water.  Hy het op sy ma se rug gebly en geskree:  "Ma, ek wil nie in die water gaan nie! Dit is te diep, te donker, te gevaarlik!"

Sy ma, ‘n pragtige leeufyn met ‘n sagte brul, het saggies gesê: "Klein Leeutjie, my liefling, water is nie gevaarlik nie. Dit is lewe. Alles in die wêreld is bang vir iets, selfs ek. Maar moed is sterker as enige vrees."

Sy het hom verduidelik dat die water vol kos is: vissies, paddas, en selfs sappige waterplante. Sy het hom gehelp om sy pote in die koel, sagte water te sit. Eers was hy bang, maar toe het die vrees begin verdwyn. Die water was nie so donker nie. Dit was meer soos ‘n groot, blink oog wat hom uitgenooi het.

‘n Klein fissie het hom met sy kolle begroet. ‘n Padda het ‘n vrolike "kerrekkerrek" gemaak. Klein Leeutjie het gelag. Hy het begin om in die water te speel, 

In [43]:
# Run a query
j = 2
moral_value = df_eval.loc[j, 'moral_value']
query_text = df_eval.loc[j, 'generated_story']

query = [query_text]

# Vectorize the query string
query_embedding = embed_model.encode(query)

# Set the number of outputs we want
top_k = 3

# Run the query
# index_vals refers to the chunk_list index values
scores, index_vals = index.search(query_embedding, top_k)

# Let's print the first match and see if
# the match and the query are identical.
# If they are identical it means the model
# is outputting stories from the train set.

pred_indexes = index_vals[0]

i = 0 # top match
chunk_index = pred_indexes[i]
train_set_story = chunk_list[chunk_index]

print('---Generated story---')
print(query_text)
print()
print('---Closest match in the train set---')
print(train_set_story)


---Generated story---
Klein Leeutjie se Groot Moed

Klein Leeutjie, met sy sagte, goue mannetjie, was bang. Bang vir die donker, bang vir die wind wat deur die gras fluister, en bo alles, bang vir die water.  Hy het op sy ma se rug gebly en geskree:  "Ma, ek wil nie in die water gaan nie! Dit is te diep, te donker, te gevaarlik!"

Sy ma, ‘n pragtige leeufyn met ‘n sagte brul, het saggies gesê: "Klein Leeutjie, my liefling, water is nie gevaarlik nie. Dit is lewe. Alles in die wêreld is bang vir iets, selfs ek. Maar moed is sterker as enige vrees."

Sy het hom verduidelik dat die water vol kos is: vissies, paddas, en selfs sappige waterplante. Sy het hom gehelp om sy pote in die koel, sagte water te sit. Eers was hy bang, maar toe het die vrees begin verdwyn. Die water was nie so donker nie. Dit was meer soos ‘n groot, blink oog wat hom uitgenooi het.

‘n Klein fissie het hom met sy kolle begroet. ‘n Padda het ‘n vrolike "kerrekkerrek" gemaak. Klein Leeutjie het gelag. Hy het begin om i

In [44]:
# Run a query
j = 1
moral_value = df_eval.loc[j, 'moral_value']
query_text = df_eval.loc[j, 'generated_story']

query = [query_text]

# Vectorize the query string
query_embedding = embed_model.encode(query)

# Set the number of outputs we want
top_k = 3

# Run the query
# index_vals refers to the chunk_list index values
scores, index_vals = index.search(query_embedding, top_k)

# Let's print the first match and see if
# the match and the query are identical.
# If they are identical it means the model
# is outputting stories from the train set.

pred_indexes = index_vals[0]

i = 0 # top match
chunk_index = pred_indexes[i]
train_set_story = chunk_list[chunk_index]

print('---Generated story---')
print(generated_story)
print()
print('---Closest match in the train set---')
print(train_set_story)


---Generated story---
Klein Karoo se Groot Dankie

Klein Karoo, ‘n klein bobbejaan met ‘n sagte, bruin pels, het in die pragtige Karoo gewoon. Hy was ‘n bietjie van ‘n verwonde bobbejaan. Hy het altyd gekla. “Ek wil ‘n nuwe baadjie hê!” sou hy huil. Of: “My avokado is te groen!”

Een warm middag, terwyl die son sy helderste was, het Karoo se ma, ‘n wyse ou bobbejaan met sterk arms, hom na ‘n rivier toe geneem. Die rivier was vol vrolike bloublomme en die water was koel en vars. Karoo het sy oë toegemaak en die son se warmte op sy vel gevoel.

Skielik het hy gesien: ‘n pragtige, blink blou klippie lê op die klippies langs die rivier. Hy het dit opgetel en dit in sy hand vasgehou. Die klippie was warmer as die son en so blink soos ‘n klein sterretjie.

“Mamma, Mamma!” het Karoo geroep. “Kyk hierdie klippie! Die mooiste klippie wat ek ooit gesien het!”

Sy ma het geglimlag. “Ja, my klein Karoo,” het sy gesê. “Dit is ‘n pragtige klippie. Maar onthou, daar is meer as net blink dinge.”

Sy h

In [45]:
# Create a requirements.txt file
!pip freeze > requirements.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
